# Discovery and Representation of Open Making Related Terms

This notebook sketches the initial exercise on discovering the open making related keywords. The input text is harvested via a Web crawler that identifies and crawls semantically related wikipedia articles.   

In [1]:
from utils import tokenizer
import nltk
from nltk import FreqDist
from nltk.stem.porter import PorterStemmer
from numpy import log, mean
import json, csv, re
import pprint as pp

import pandas as pd

## 1. Loading a reference English language corpus

In [2]:
from nltk.corpus import brown
brown.categories()

[u'adventure',
 u'belles_lettres',
 u'editorial',
 u'fiction',
 u'government',
 u'hobbies',
 u'humor',
 u'learned',
 u'lore',
 u'mystery',
 u'news',
 u'religion',
 u'reviews',
 u'romance',
 u'science_fiction']

## 2. Stop words

### 2.1 Standard stop words

In [3]:
with open("data/stopwords_standard.txt", "r") as f:
    STOP_WORDS_STANDARD = set(f.read().strip().split("\n"))
print(STOP_WORDS_STANDARD)

set(['all', "she'll", 'just', "don't", 'being', 'over', 'through', 'yourselves', 'its', 'before', "he's", "when's", "we've", 'had', 'should', "he'd", 'to', 'only', "there's", 'those', 'under', 'has', "haven't", 'do', 'them', 'his', "they'll", 'get', 'very', "who's", "they'd", 'cannot', 'know', 'they', 'not', 'during', 'yourself', 'him', 'nor', "we'll", 'like', 'did', "they've", 'this', 'she', 'each', "won't", 'where', "mustn't", "isn't", "i'll", "why's", 'www', 'because', "you'd", 'doing', 'some', 'up', 'are', 'further', 'ourselves', 'out', 'what', 'for', 'while', "wasn't", 'does', "shouldn't", 'above', 'between', 'ought', 'be', 'we', 'who', "you're", 'were', 'here', 'hers', "aren't", 'by', 'both', 'about', 'would', 'of', 'could', "i'd", "weren't", "i'm", 'com', 'or', "can't", 'own', 'into', 'whom', 'down', "hadn't", "couldn't", 'your', "doesn't", 'from', "how's", 'her', 'their', "it's", 'there', 'been', 'why', 'few', 'too', 'themselves', 'was', 'until', 'more', 'himself', "where's", "

### 2.2 Open-making related stop words

In [4]:
with open("data/stopwords_openmaker.txt", "r") as f:
    STOP_WORDS_OPENMAKER = set(f.read().strip().split("\n"))
print(STOP_WORDS_OPENMAKER)

set(['isbn', 'often', 'vol', 'vi', 'eg', 'one', 'ii', 'second', '1st', '7th', '11th', 'txt', 'ad', 'pp', '6th', '3rd', 'na', '5th', 'wikipedia', 'randd', '14th', 'also', 'html', 'von', '15th', 'first', 'bc', 'may', '4th', 'wikipedias', 'org', 'iv', 'iii', '13th', 'almost', 'doi', 'third', 'many', 'well', 'britannica', '2nd', 'etc', 'encyclopedia', '9th', 'doc', 'pdf', '10th', 'tt', '12th', '8th'])


## 3. Removing stop words from the reference English corpus

In [5]:
# merging the two list together
STOP_WORDS = STOP_WORDS_STANDARD.union(STOP_WORDS_OPENMAKER)
print(STOP_WORDS)

set(['all', "she'll", 'just', "don't", 'being', 'over', 'through', 'during', 'once', 'its', 'before', "he's", "when's", "we've", 'tt', 'had', 'html', 'randd', 'should', "he'd", 'to', 'only', 'does', "here's", 'under', 'has', "haven't", 'do', 'them', 'his', 'above', 'get', 'very', "who's", "they'd", 'cannot', 'know', 'they', 'not', 'yourselves', 'one', 'him', 'nor', "we'll", 'like', 'did', '12th', "they've", "wasn't", 'she', 'each', "won't", 'where', "mustn't", "isn't", "i'll", "why's", 'www', 'because', "you'd", 'doing', 'eg', 'theirs', 'some', "hasn't", 'second', 'are', '7th', 'further', '11th', 'ourselves', 'out', 'what', 'for', 'herself', 'bc', 'wikipedia', 'below', '14th', 'may', "there's", "shouldn't", "they'll", 'between', '15th', 'can', 'be', 'we', 'after', "doesn't", 'doc', 'here', 'hers', 'org', "aren't", 'by', 'von', 'both', 'about', 'her', '8th', 'of', 'could', 'britannica', 'etc', "i'd", "weren't", 'pdf', "i'm", 'com', 'or', "can't", 'first', 'own', 'isbn', 'into', 'yoursel

In [6]:
# load english words from the Brown corpus removing stop words.
english_freq_dist = FreqDist([w.lower() for w in nltk.corpus.brown.words()
                              if w not in STOP_WORDS])
for stopword in STOP_WORDS:
    if stopword in english_freq_dist:
        del english_freq_dist[stopword]
        
for punctuation in tokenizer.CHARACTERS_TO_SPLIT:
    if punctuation in english_freq_dist:
        del english_freq_dist[punctuation]

# Re-control most common words after cleaning:
n_english = sum(english_freq_dist.values())
english_freq_dist.most_common(10)

[(u'``', 8837),
 (u"''", 8789),
 (u'--', 3432),
 (u'will', 2245),
 (u'said', 1961),
 (u'new', 1635),
 (u'time', 1598),
 (u'two', 1412),
 (u'now', 1314),
 (u'man', 1207)]

## 4. Loading the input Open Maker corpus

In [7]:
# load the harvested text from wikipedia.
with open("data/wikipedia2.json", "r") as f: OM_Corpus_text = f.read()
OM_Corpus = json.loads(OM_Corpus_text)

In [8]:
# The total number of wiki articles used:
print(len(OM_Corpus))

144


In [9]:
# Column names of the the corpus.
OM_Corpus[0].keys()

[u'url', u'text', u'depth', u'theme.id', u'title']

In [10]:
def display_articles(tid):
    articles = [article for article in OM_Corpus if article['theme.id'] == tid]
    for article in articles:
        print(article['depth'],article['title'], article['url'])

In [11]:
display_articles(1)

(0, u'Power (social and political)', u'https://en.wikipedia.org/wiki/Power_(social_and_political)')
(1, u'Speaking truth to power', u'https://en.wikipedia.org/wiki/Speaking_truth_to_power')
(1, u'State collapse', u'https://en.wikipedia.org/wiki/State_collapse')
(1, u'The Anatomy of Revolution', u'https://en.wikipedia.org/wiki/The_Anatomy_of_Revolution')
(1, u'Authority bias', u'https://en.wikipedia.org/wiki/Authority_bias')
(1, u'Discourse of power', u'https://en.wikipedia.org/wiki/Discourse_of_power')
(1, u'Chronemics', u'https://en.wikipedia.org/wiki/Control_of_time_in_power_relationships')
(1, u'Personal boundaries', u'https://en.wikipedia.org/wiki/Personal_boundaries')
(1, u'Cratology', u'https://en.wikipedia.org/wiki/Cratology')
(1, u'Veto', u'https://en.wikipedia.org/wiki/Veto')
(1, u'Amity-enmity complex', u'https://en.wikipedia.org/wiki/Amity-enmity_complex')
(1, u'Social control', u'https://en.wikipedia.org/wiki/Social_control')


In [12]:
display_articles(2)

(0, u'Social influence', u'https://en.wikipedia.org/wiki/Social_influence')
(0, u'Need for achievement', u'https://en.wikipedia.org/wiki/Need_for_achievement')
(1, u'Goal orientation', u'https://en.wikipedia.org/wiki/Goal_orientation')
(1, u'Need for cognition', u'https://en.wikipedia.org/wiki/Need_for_cognition')
(1, u'Need for power', u'https://en.wikipedia.org/wiki/Need_for_power')
(1, u'Need theory', u'https://en.wikipedia.org/wiki/Need_theory')
(1, u'Need for affiliation', u'https://en.wikipedia.org/wiki/Need_for_affiliation')
(1, u'Bystander effect', u'https://en.wikipedia.org/wiki/Bystander_effect')
(1, u'Social proof', u'https://en.wikipedia.org/wiki/Social_proof')
(1, u'Mind shaping', u'https://en.wikipedia.org/wiki/Mind_shaping')
(1, u'Propaganda', u'https://en.wikipedia.org/wiki/Propaganda')
(1, u'Judge\u2013advisor system', u'https://en.wikipedia.org/wiki/Judge%E2%80%93advisor_system')
(1, u'Impression management', u'https://en.wikipedia.org/wiki/Impression_management')


In [13]:
display_articles(3)

(0, u'Hedonism', u'https://en.wikipedia.org/wiki/Hedonism')
(1, u'Utilitarianism', u'https://en.wikipedia.org/wiki/Utilitarianism')
(1, u'Paradox of hedonism', u'https://en.wikipedia.org/wiki/Paradox_of_hedonism')
(1, u'Epicureanism', u'https://en.wikipedia.org/wiki/Epicureanism')
(1, u'Eudaimonia', u'https://en.wikipedia.org/wiki/Eudaimonia')
(1, u'Yangism', u'https://en.wikipedia.org/wiki/Yangism')
(1, u'Pleasure principle (psychology)', u'https://en.wikipedia.org/wiki/Pleasure_principle_(psychology)')
(1, u'Cyrenaics', u'https://en.wikipedia.org/wiki/Cyrenaics')
(1, u'Torbj\xf6rn T\xe4nnsj\xf6', u'https://en.wikipedia.org/wiki/Torbj%C3%B6rn_T%C3%A4nnsj%C3%B6')
(1, u'Hedonism Resorts', u'https://en.wikipedia.org/wiki/Hedonism_Resorts')
(1, u'Psychological egoism', u'https://en.wikipedia.org/wiki/Psychological_hedonism')
(1, u'Michel Onfray', u'https://en.wikipedia.org/wiki/Michel_Onfray')
(1, u'David Pearce (philosopher)', u'https://en.wikipedia.org/wiki/David_Pearce_(philosopher)')


In [14]:
display_articles(4)

(0, u'Stimulation', u'https://en.wikipedia.org/wiki/Stimulation')


In [15]:
display_articles(5)

(0, u'Independence', u'https://en.wikipedia.org/wiki/Independence')
(0, u'Freedom', u'https://en.wikipedia.org/wiki/Freedom')
(0, u'Creativity', u'https://en.wikipedia.org/wiki/Creativity')


In [16]:
display_articles(6)

(0, u'Universalism', u'https://en.wikipedia.org/wiki/Universalism')
(0, u'Social justice', u'https://en.wikipedia.org/wiki/Social_justice')
(0, u'Egalitarianism', u'https://en.wikipedia.org/wiki/Egalitarianism')
(0, u'Environmental protection', u'https://en.wikipedia.org/wiki/Environmental_protection')


In [17]:
display_articles(7)

(0, u'Loyalty', u'https://en.wikipedia.org/wiki/Loyalty')
(0, u'Altruism', u'https://en.wikipedia.org/wiki/Altruism')
(0, u'Responsibility', u'https://en.wikipedia.org/wiki/Responsibility')
(1, u'Alms', u'https://en.wikipedia.org/wiki/Alms')
(1, u'Inclusive fitness', u'https://en.wikipedia.org/wiki/Inclusive_fitness')
(1, u'Tragedy of the commons', u'https://en.wikipedia.org/wiki/Comedy_of_the_commons')
(1, u"Prisoner's dilemma", u'https://en.wikipedia.org/wiki/Prisoner%27s_dilemma')
(1, u'Egotism', u'https://en.wikipedia.org/wiki/Egotism')
(1, u'Kin selection', u'https://en.wikipedia.org/wiki/Kin_selection')
(1, u'Empathy-altruism', u'https://en.wikipedia.org/wiki/Empathy-altruism')
(1, u'Prosocial behavior', u'https://en.wikipedia.org/wiki/Prosocial_behavior')
(1, u'Mutual aid (organization theory)', u'https://en.wikipedia.org/wiki/Mutual_aid_(organization)')
(1, u'Solidarity', u'https://en.wikipedia.org/wiki/Solidarity_(sociology)')
(1, u'Social psychology', u'https://en.wikipedia.o

In [18]:
display_articles(8)

(0, u'Tradition', u'https://en.wikipedia.org/wiki/Tradition')
(0, u'Modesty', u'https://en.wikipedia.org/wiki/Modesty')
(1, u'Perennial philosophy', u'https://en.wikipedia.org/wiki/Perennial_philosophy')
(1, u'Folklore', u'https://en.wikipedia.org/wiki/Folklore')
(1, u'Origin myth', u'https://en.wikipedia.org/wiki/Aition')


In [19]:
display_articles(9)

(0, u'Conformity', u'https://en.wikipedia.org/wiki/Conformity')
(1, u'Cultural assimilation', u'https://en.wikipedia.org/wiki/Cultural_assimilation')
(1, u'Authoritarian personality', u'https://en.wikipedia.org/wiki/Authoritarian_personality')
(1, u'Spiral of silence', u'https://en.wikipedia.org/wiki/Spiral_of_silence')
(1, u'Milieu control', u'https://en.wikipedia.org/wiki/Milieu_control')
(1, u'Countersignaling', u'https://en.wikipedia.org/wiki/Countersignaling')
(0, u'Obedience (human behavior)', u'https://en.wikipedia.org/wiki/Obedience_(human_behavior)')
(0, u'Discipline', u'https://en.wikipedia.org/wiki/Discipline')
(1, u'Social norm', u'https://en.wikipedia.org/wiki/Norm_(social)')
(1, u'Deindividuation', u'https://en.wikipedia.org/wiki/Deindividuation')
(1, u'School discipline', u'https://en.wikipedia.org/wiki/School_discipline')
(1, u'Positive discipline', u'https://en.wikipedia.org/wiki/Positive_discipline')
(1, u'Child discipline', u'https://en.wikipedia.org/wiki/Child_disci

In [20]:
display_articles(10)

(0, u'Security', u'https://en.wikipedia.org/wiki/Security')
(0, u'Social order', u'https://en.wikipedia.org/wiki/Social_order')
(0, u'Cleanliness', u'https://en.wikipedia.org/wiki/Cleanliness')
(1, u'Cleanroom', u'https://en.wikipedia.org/wiki/Clean_room')
(1, u'Green cleaning', u'https://en.wikipedia.org/wiki/Green_cleaning')
(1, u'Cleaning', u'https://en.wikipedia.org/wiki/Cleaning')
(1, u'Waste management', u'https://en.wikipedia.org/wiki/Waste_management')
(1, u'Environmental remediation', u'https://en.wikipedia.org/wiki/Environmental_remediation')
(1, u'Pollution', u'https://en.wikipedia.org/wiki/Pollution')
(1, u'Contamination control', u'https://en.wikipedia.org/wiki/Contamination_control')
(1, u'Lady Macbeth effect', u'https://en.wikipedia.org/wiki/Lady_Macbeth_effect')
(1, u'Antiseptic', u'https://en.wikipedia.org/wiki/Antiseptic')
(1, u'Hygiene', u'https://en.wikipedia.org/wiki/Hygiene')
(1, u'Cleaner', u'https://en.wikipedia.org/wiki/Cleaner')
(1, u'Ritual purification', u'h

## 5. Analyzing a specific corpus based on a theme

In [21]:
def get_title(Corpus, theme_id):
    title = ''
    for article in Corpus:
        if article['theme.id'] == theme_id:
            title = article['title']
            break
    return title

## 6.0 Selecting the specific theme (a sub-corpus).

In [22]:
## For a different sub-corpus use a corresponding theme ID.
current_theme_id = 6

In [23]:
current_title = get_title(OM_Corpus, current_theme_id)

In [24]:
output_fname = "_".join([word.capitalize() for word in current_title.split(" ")])
print(current_title, "::", output_fname)

(u'Universalism', '::', u'Universalism')


In [25]:
input_text = " ".join([page['text'] for page in OM_Corpus if page['theme.id'] == current_theme_id])

In [26]:
pp.pprint(input_text)

u'Universalism \n For other uses see \n Universalism disambiguation \n Part of \n a series \n on \n Universalism \n Philosophical \n Moral universalism \n Universality \n Universalizability \n Christian \n Beliefs \n Apocatastasis \n Theosis \n Trinitarian Universalism \n Universal reconciliation \n Churches and groups \n Unitarian Universalism \n Christian Universalist Association \n International Council of \n Unitarians and Universalists \n Primitive Baptist Universalists \n Universalist Church of America \n Other religions \n Bah\' Faith \n Hinduism \n Neo-Vedanta \n Sufism \n Spiritual \n Mysticism \n Nondualism \n Perennialism \n Theosophy \n Category \n v \n t \n e \n Universalism \n is a theological and philosophical concept that some ideas have \n universal \n application or applicability \n citation needed \n A community that calls itself \n universalist \n may emphasize the universal principles of most religions and accept other religions in an inclusive manner believing in 

In [27]:
# Tokenizing the input text:
tokenized = tokenizer.tokenize_words(input_text)
number_of_words = len(tokenized)
print(number_of_words,current_title)

(20477, u'Universalism')


### 6.1 Computing frequency distributions of each token, i.e word, term, pancuation, etc.

In [28]:
input_freq_dist = FreqDist(tokenized)

In [29]:
input_freq_dist.most_common(10)

[(u'\n', 2890),
 (u'the', 1009),
 (u'of', 809),
 (u'and', 606),
 (u'in', 375),
 (u'to', 337),
 (u'a', 266),
 (u'that', 204),
 (u'is', 203),
 (u'as', 182)]

### 6.2 Removing punctuation and stopwords from the input corpus

In [30]:
for stopword in STOP_WORDS:
    if stopword in input_freq_dist:
        del input_freq_dist[stopword]
        
for punctuation in tokenizer.CHARACTERS_TO_SPLIT:
    if punctuation in input_freq_dist:
        del input_freq_dist[punctuation]

# Re-control most common words after cleaning:
n_input = sum(input_freq_dist.values())
input_freq_dist.most_common(10)

[(u'social', 161),
 (u'justice', 134),
 (u'environmental', 109),
 (u's', 79),
 (u'"', 72),
 (u'rights', 59),
 (u'human', 58),
 (u'protection', 57),
 (u'environment', 53),
 (u'people', 47)]

### 6.3 Removing all numbered words

This is an example case for post proceesing in terms of cleaning. The pre-processing, that is data cleaning/preperation during or right after harvesting should be further improved to avoid such processes at this stage.

In [31]:
pattern_letters = re.compile('[a-z]')
def has_letters(x):
    return(pattern_letters.search(x) is not None)

In [32]:
reduced = {k:v for k,v in input_freq_dist.items() if has_letters(k)}
print("Reduction due to all number matches: ", len(input_freq_dist) - len(reduced))
input_freq_dist = reduced

('Reduction due to all number matches: ', 44)


### 6.4 Removing single character words


In [33]:
reduced = {k:v for k,v in input_freq_dist.items() if len(k) > 1}
print("Reduction due to single characters: ", len(input_freq_dist) - len(reduced))
input_freq_dist = reduced

('Reduction due to single characters: ', 20)


### 6.5 Removing rare words from input distribution

In [34]:
reduced = {k:v for k,v in input_freq_dist.items() if v > 5}
print("Reduction due to rare occurances: ", len(input_freq_dist) - len(reduced))
input_freq_dist = reduced

('Reduction due to rare occurances: ', 3578)


## 7. Comparing input vs English corpus volumes

### 7.1 Total words (after cleaning the stopwords) 

In [35]:
print(n_input, n_english)

(10922, 544168)


### 7.2 Number of unique words (after cleaning stopwords and rare words)

In [36]:
n_unique_word_input = len(input_freq_dist.items())
n_unique_word_brown = len(english_freq_dist.items())
n_unique_word_input, n_unique_word_brown

(388, 49598)

### 7.3 Cleaned set of input words/terms

List of words in the corpus in case, for a visual inspection. Such inspections will be used both to improve tokenization as well as filtering.

In [37]:
pp.pprint(sorted(input_freq_dist.items(), key=lambda x:x[1], reverse=True))

[(u'social', 161),
 (u'justice', 134),
 (u'environmental', 109),
 (u'rights', 59),
 (u'human', 58),
 (u'protection', 57),
 (u'environment', 53),
 (u'people', 47),
 (u'god', 46),
 (u'egalitarianism', 45),
 (u'universalism', 45),
 (u'equality', 42),
 (u'world', 40),
 (u'united', 38),
 (u'john', 38),
 (u'states', 36),
 (u'society', 35),
 (u'national', 34),
 (u'economic', 33),
 (u'christian', 32),
 (u'men', 31),
 (u'international', 29),
 (u'moral', 29),
 (u'religions', 29),
 (u'women', 28),
 (u'movement', 27),
 (u'church', 27),
 (u'religious', 26),
 (u'state', 26),
 (u'universal', 25),
 (u'development', 24),
 (u'theory', 23),
 (u'countries', 23),
 (u'political', 23),
 (u'equal', 23),
 (u'theology', 23),
 (u'issues', 22),
 (u'islam', 22),
 (u'health', 22),
 (u'new', 22),
 (u'religion', 22),
 (u'pollution', 22),
 (u'universalist', 22),
 (u'law', 21),
 (u'principles', 21),
 (u'use', 21),
 (u'liberal', 21),
 (u'china', 20),
 (u'catholic', 20),
 (u'america', 20),
 (u'conservation', 19),
 (u'lib

### 7.4 Set of terms/words that occure in both corpus.

In [38]:
len(input_freq_dist.keys())

388

In [39]:
common_words = [w for w in set(input_freq_dist.keys()) & set(english_freq_dist.keys())]
print(len(common_words))

366


In [40]:
pp.pprint(sorted(common_words))

[u'20th',
 u'according',
 u'achieve',
 u'act',
 u'action',
 u'activities',
 u'activity',
 u'africa',
 u'against',
 u'age',
 u'agencies',
 u'agreements',
 u'air',
 u'al',
 u'allah',
 u'although',
 u'america',
 u'american',
 u'among',
 u'animals',
 u'approach',
 u'area',
 u'areas',
 u'argued',
 u'article',
 u'articles',
 u'australia',
 u'bah',
 u'based',
 u'basic',
 u'became',
 u'become',
 u'began',
 u'beings',
 u'belief',
 u'beliefs',
 u'believe',
 u'book',
 u'books',
 u'brazil',
 u'broad',
 u'called',
 u'care',
 u'catholic',
 u'central',
 u'century',
 u'change',
 u'china',
 u'christ',
 u'christian',
 u'christianity',
 u'church',
 u'citation',
 u'citizen',
 u'citizens',
 u'class',
 u'common',
 u'communities',
 u'community',
 u'concept',
 u'concepts',
 u'conservation',
 u'conservative',
 u'considered',
 u'constitution',
 u'contract',
 u'control',
 u'council',
 u'countries',
 u'country',
 u'created',
 u'cultural',
 u'current',
 u'day',
 u'de',
 u'decision',
 u'declaration',
 u'definition'

### 7.5 Set of terms/words that occure in the sample but not in the reference corpus.

This specific set will be incorporated later below. 

In [41]:
input_specifics = dict()
for w in set(input_freq_dist.keys()) - set(english_freq_dist.keys()):
    input_specifics[w] = input_freq_dist[w]

In [42]:
print(len(input_specifics))

22


In [43]:
pp.pprint(sorted(input_specifics))

[u'"social',
 u'"the',
 u'archived',
 u'biodiversity',
 u'ecology',
 u'ecosystems',
 u'egalitarian',
 u'feminism',
 u'gdp',
 u'justice"',
 u'pogge',
 u'programme',
 u'rawls',
 u'sikh',
 u'sikhism',
 u'sustainable',
 u'tanzania',
 u'universalism',
 u'universalist',
 u'universalists',
 u'vedanta',
 u'wayback']


## 8. Stemming

In [44]:
stemmer = PorterStemmer()
input_wset_stems = {k: stemmer.stem(k) for k in input_freq_dist.keys()}
common_wset_stems = {k: stemmer.stem(k) for k in common_words}
pp.pprint(common_wset_stems)

{u'20th': u'20th',
 u'according': u'accord',
 u'achieve': u'achiev',
 u'act': u'act',
 u'action': u'action',
 u'activities': u'activ',
 u'activity': u'activ',
 u'africa': u'africa',
 u'against': u'against',
 u'age': u'age',
 u'agencies': u'agenc',
 u'agreements': u'agreement',
 u'air': u'air',
 u'al': u'al',
 u'allah': u'allah',
 u'although': u'although',
 u'america': u'america',
 u'american': u'american',
 u'among': u'among',
 u'animals': u'anim',
 u'approach': u'approach',
 u'area': u'area',
 u'areas': u'area',
 u'argued': u'argu',
 u'article': u'articl',
 u'articles': u'articl',
 u'australia': u'australia',
 u'bah': u'bah',
 u'based': u'base',
 u'basic': u'basic',
 u'became': u'becam',
 u'become': u'becom',
 u'began': u'began',
 u'beings': u'be',
 u'belief': u'belief',
 u'beliefs': u'belief',
 u'believe': u'believ',
 u'book': u'book',
 u'books': u'book',
 u'brazil': u'brazil',
 u'broad': u'broad',
 u'called': u'call',
 u'care': u'care',
 u'catholic': u'cathol',
 u'central': u'centra

## 9. Handling input specific term set

### 9.1 Identfying matching stems with common words. 

Note that the frequency counts are transferred accordingly.

In [45]:
specifics = {}
for k,v in input_specifics.items():
    stem = input_wset_stems[k]
    words = [w for w,s in common_wset_stems.items() if s == stem]
    if words:
        w = words[0]
        input_freq_dist[w] += input_specifics[k]
        continue
    specifics[k] = v
# Removing the words with matching stems from the specific set.
print("Reduction due to stemm matches: ", len(input_specifics) - len(specifics))
input_specifics = specifics

('Reduction due to stemm matches: ', 3)


### 9.2 Removing open-maker specific terms.

In [46]:
with open("data/specifics_openmaker.txt", "r") as f:
    SPECIFICS_OPENMAKER = set(f.read().strip().split("\n"))
om_specific_stems = {stemmer.stem(k):k for k in SPECIFICS_OPENMAKER}
pp.pprint(om_specific_stems)

{'3-d': '3-d',
 '3d': '3d',
 u'abat': 'abatement',
 u'afford': 'affordable',
 'agenda21': 'agenda21',
 u'anarch': 'anarchism',
 u'autonom': 'autonomous',
 'biodiesel': 'biodiesel',
 u'biodivers': 'biodiversity',
 'biofuel': 'biofuel',
 u'bioga': 'biogas',
 u'biomass': 'biomass',
 u'biospher': 'biosphere',
 u'bricolag': 'bricolage',
 'brundtland': 'brundtland',
 'c2c': 'c2c',
 'cad': 'cad',
 u'cap-and-trad': 'cap-and-trade',
 u'carfre': 'carfree',
 'cdm': 'cdm',
 'christensen': 'christensen',
 u'co-creat': 'co-creation',
 'co-develop': 'co-develop',
 u'co-invent': 'co-invention',
 'co-inventor': 'co-inventor',
 u'coextinct': 'coextinction',
 u'cognit': 'cognition',
 u'commons-bas': 'commons-based',
 u'computer-aid': 'computer-aided',
 u'conferenc': 'conferencing',
 'consortium': 'consortium',
 u'constraint': 'constraints',
 'construct': 'construct',
 'copyleft': 'copyleft',
 'copyright': 'copyright',
 u'cradle-to-cradl': 'cradle-to-cradle',
 u'crowdsourc': 'crowdsourcing',
 u'crowdwork'

In [47]:
specific_wset_dirty = {}
specific_wset_stems_selected = {}
for k,v in input_specifics.items():
    stem = stemmer.stem(k)
    if stem in om_specific_stems.keys():
        if k in specific_wset_stems_selected.keys():
            specific_wset_stems_selected[stem] += v
        else:
            specific_wset_stems_selected[stem] = v
        continue
    if v > 10:
        specific_wset_dirty[k] = v
        
input_specifics = specific_wset_dirty

In [48]:
# The set of stems to be added to the set with makerness counts.

pp.pprint(specific_wset_stems_selected)

{u'biodivers': 7, u'ecosystem': 8, u'programm': 7, u'sustain': 11}


### 9.3 Remaining frequent input specifics
The manual checking can help to determine what should go into "specifics_openmaker.txt"

In [49]:
print(len(input_specifics))
pp.pprint(sorted(input_specifics))

7
[u'"the',
 u'archived',
 u'feminism',
 u'justice"',
 u'rawls',
 u'universalist',
 u'universalists']


In [50]:
specific_wset_stems = {}
for k,v in input_specifics.items():
    stem = stemmer.stem(k)
    if stem in specific_wset_stems.keys():
        specific_wset_stems[stem].append((k,v))
    else:
        specific_wset_stems[stem] = [(k,v)]
pp.pprint(specific_wset_stems)

{u'"the': [(u'"the', 12)],
 u'archiv': [(u'archived', 18)],
 u'femin': [(u'feminism', 18)],
 u'justice"': [(u'justice"', 11)],
 u'rawl': [(u'rawls', 16)],
 u'universalist': [(u'universalist', 22), (u'universalists', 11)]}


## 10. Computing representation power of common words.

In [51]:
# combine
word = []
freq = []
score = []
# common_words = [w[0] for w in common_words]
nEng = 1.0 * n_english
nInp = 1.0 * n_input
for w in common_words:
    # Consider only words whose charcater length is larger than 1
    if len(w) > 1:
        # Log likelihood scores are computed:
        print(w, input_freq_dist[w], english_freq_dist[w])
        s = log((input_freq_dist[w] / nInp) / (english_freq_dist[w] / nEng))
        f = input_freq_dist[w]
        word.append(w)
        freq.append(f)
        score.append(s)

(u'concept', 16, 85)
(u'global', 13, 4)
(u'mill', 6, 11)
(u'religious', 26, 165)
(u'issues', 22, 66)
(u'environment', 53, 43)
(u'must', 13, 1013)
(u'teaching', 6, 67)
(u'include', 8, 113)
(u'resources', 12, 72)
(u'activities', 7, 115)
(u'regional', 7, 42)
(u'every', 9, 491)
(u'word', 7, 274)
(u'governments', 9, 61)
(u'judaism', 13, 2)
(u'level', 12, 213)
(u'gender', 15, 2)
(u'list', 7, 133)
(u'race', 6, 103)
(u'20th', 6, 12)
(u'spiritual', 7, 64)
(u'says', 7, 200)
(u'literature', 8, 133)
(u'perspective', 8, 26)
(u'theory', 23, 129)
(u'even', 6, 1170)
(u'established', 10, 108)
(u'constitution', 7, 49)
(u'section', 6, 189)
(u'liberty', 11, 46)
(u'current', 6, 104)
(u'template', 6, 5)
(u'international', 29, 155)
(u'egalitarianism', 57, 2)
(u'public', 11, 438)
(u'movement', 27, 128)
(u'christian', 32, 144)
(u'men', 31, 763)
(u'french', 7, 139)
(u'understanding', 7, 121)
(u'water', 6, 445)
(u'protection', 57, 68)
(u'china', 20, 69)
(u'groups', 15, 125)
(u'strong', 6, 202)
(u'teaches', 7, 11

In [52]:
m = pd.Series(score)
f = pd.Series(freq)
k = pd.Series(word)
stem = k.apply(stemmer.stem)
scoring = len(k) * ['common']
df_common = pd.DataFrame({'Word':k, 'Stem':stem, 'Score':m,'Tf':f, 'Type':scoring})
df_common.sort_values(by='Score', ascending=False, inplace = True)
df_common.reset_index(inplace=True, drop=True)
df_common.head(20)

,Score,Stem,Tf,Type,Word
0,7.258383,egalitarian,57,common,egalitarianism
1,6.741692,univers,51,common,universality
2,6.653917,environment,109,common,environmental
3,6.473428,feminist,13,common,feminist
4,6.211064,bah,10,common,bah
5,6.048545,hinduism,17,common,hinduism
6,5.987920,allah,8,common,allah
7,5.923382,gender,15,common,gender
8,5.900909,islam,22,common,islam
9,5.780281,ecolog,13,common,ecological


## Computing makerness of specific terms

In [53]:
max_score = df_common.iloc[0,0]
threshold_score = 1.0
mean_w = df_common.Score[df_common.Score > threshold_score].mean()
mean_f = df_common.Tf[df_common.Score > threshold_score].mean()
print(max_score, threshold_score, mean_w, mean_f)

(7.2583830062538262, 1.0, 2.4526788452794723, 14.94238683127572)


In [54]:
def compute_speficif_score(f, maxw=10, minw=1, mean_w=3.0, mean_f=25.0):
    return(max(min((f / mean_f) * mean_w, maxw), minw))
compute_speficif_score(mean_w, max_score, threshold_score, mean_w, mean_f)

1.0

In [55]:
stem = []
freq = []
score = []
for k, v in specific_wset_stems_selected.items():
    s = compute_speficif_score(v, max_score, threshold_score, mean_w, mean_f)
    stem.append(k)
    freq.append(v)
    score.append(s)

In [56]:
m = pd.Series(score)
f = pd.Series(freq)
stem = pd.Series(stem)
scoring = len(m) * ['specific']
df_exclusive= pd.DataFrame({'Stem':stem, 'Score':m,'Tf':f, 'Type':scoring})
df_exclusive.sort_values(by='Score', ascending=False, inplace = True)
df_exclusive.reset_index(inplace=True, drop=True)
df_exclusive.head(20)

,Score,Stem,Tf,Type
0,1.805566,sustain,11,specific
1,1.313139,ecosystem,8,specific
2,1.148997,biodivers,7,specific
3,1.148997,programm,7,specific


In [57]:
df_makerness = df_common.append(df_exclusive, ignore_index=True)
df_makerness.sort_values(by='Score', ascending=False, inplace = True)
df_makerness.reset_index(inplace=True, drop=True)
df_makerness.head(42)

,Score,Stem,Tf,Type,Word
0,7.258383,egalitarian,57,common,egalitarianism
1,6.741692,univers,51,common,universality
2,6.653917,environment,109,common,environmental
3,6.473428,feminist,13,common,feminist
4,6.211064,bah,10,common,bah
5,6.048545,hinduism,17,common,hinduism
6,5.987920,allah,8,common,allah
7,5.923382,gender,15,common,gender
8,5.900909,islam,22,common,islam
9,5.780281,ecolog,13,common,ecological


In [58]:
df_makerness.tail(42)

,Score,Stem,Tf,Type,Word
328,0.055063,right,13,common,right
329,0.044247,whether,6,common,whether
330,0.003984,later,8,common,later
331,0.000464,gener,10,common,general
332,-0.003544,time,6,common,times
333,-0.028933,becom,7,common,become
334,-0.037506,american,11,common,american
335,-0.037946,set,8,common,set
336,-0.042765,thu,6,common,thus
337,-0.042765,began,6,common,began


In [59]:
OUTPUT_FOLDER = "./output/"
csvfile_name = OUTPUT_FOLDER + "makerness_" + output_fname + ".csv"
with open(csvfile_name, 'w') as csvfile:
    #thewriter = csv.writer(csvfile, delimiter=',')
    #for k,v in sorted(makerness.items(), key=lambda x:x[1][0], reverse=True): thewriter.writerow([k,v[0],v[1]])
    df_makerness.to_csv(csvfile_name)

In [60]:
print(csvfile_name)

./output/makerness_Universalism.csv
